# Get a Freaggin AST

In [21]:
import pandas as pd
import numpy as np
import os

In [2]:
def prepare(output):
    linesList = output.line.value_counts().index.sort_values()[1:]
    ast = [['TEMPORARY']]
    flag_open = False
    flag_close = False
    for linenum in linesList:
        statement = []
        line = output[output.line==linenum]
        leaf = line[line.type=='LEAF_NODE'].code.values

        chk_var_dec = (line.type=='VAR_DECL').tolist()
        if True in chk_var_dec:
            vardec = np.where(chk_var_dec)[0]
            for item in vardec:
                statement = pd.Series(line.code.tolist()[item].split(' '))
                statement = statement[statement!=';']
                statement = statement[statement!='=']
                if True in (line.type=='POINTER').tolist():
                    statement = statement.tolist()
                    statement.insert(0,'Pointer')
                    statement.insert(0,'Decl')
                else:
                    statement = statement.tolist()
                    statement.insert(0,'Decl')
                ast.append(pd.Series(statement).unique().tolist())
            continue

        if linenum == linesList[0]:
            statement.append('FuncDef')
            statement.append('Decl')
            statement.append('FuncDecl')

        if '{' in leaf:
            flag_open = True

        leaf = leaf[leaf!='{']
        leaf = leaf[leaf!=';']
        leaf = leaf[leaf!='(']
        leaf = leaf[leaf!=')']
        leaf = leaf[leaf!=',']

        for item in leaf:
            item = 'Return' if item == 'return' else item
            if item != item:
                continue
            else:
                item
            statement.append(item)


        if len(statement)!=0:

            if True in (line.type=='POINTER').tolist():
                statement.insert(0,'Pointer')

            if True in (line.type=='SELECTION').tolist():
                statement.insert(0,'Condition')
                statement.insert(0,'Selection')

            if True in (line.type=='ITERATION').tolist():
                statement.insert(0,'ForInit')
                statement.insert(0,'Condition')
                statement.insert(0,'ForExpr')            
                statement.insert(0,'Iteration')

            if True in (line.type=='FUNCTION_CALL').tolist():
                statement.insert(0,'ExprList')
                statement.insert(0,'FuncCall')
                if statement == ast[-1]:
                    continue

                ast.append(statement)

            else:
                if statement == ast[-1]:
                    continue
                ast.append(statement)

        if flag_open == True:
            ast.append(['Compound'])
            flag_open = False

        if '}' in leaf:
            del(ast[-1])
            ast.append(['End'])

    del(ast[0])
    return(ast)

In [3]:
## TRUE (Change directory)

mydir = 'train/true'
df_true = pd.DataFrame(columns=['id', 'code', 'label'])
c_id = 0
for file in os.listdir(mydir):
    try:
        mydf = pd.read_csv(mydir+'/'+file,sep='\t',names=['type','line','level','code'], error_bad_lines = False,engine='python',warn_bad_lines=False)
        df_true = df_true.append({'id': c_id, 'code': prepare(mydf), 'label': 1}, ignore_index=True)
        c_id += 1
    except:
        print(file)
        break

In [ ]:
## FALSE (Change directory)

mydir = 'train/false'
df_false = pd.DataFrame(columns=['id', 'code', 'label'])
for file in os.listdir(mydir):
    mydf = pd.read_csv(mydir+'/'+file,sep='\t',names=['type','line','level','code'], error_bad_lines = False,engine='python',warn_bad_lines=False)
    df_false = df_false.append({'id': c_id, 'code': prepare(mydf), 'label': 0}, ignore_index=True)
    c_id += 1

In [6]:
## Combine (Change filename)

all = pd.concat([df_true,df_false])
all = all.sample(frac=1, random_state=666)
all.to_pickle('train/train_name_block.pkl')

In [5]:
df_false

,id,code,label
0,65904,"[[Pointer, FuncDef, Decl, FuncDecl, array_prin...",0
1,65905,"[[Decl, int, Int1], [Decl, int, L], [Decl, int...",0
2,65906,"[[Pointer, FuncDef, Decl, FuncDecl, skip_space...",0
3,65907,"[[Decl, const, long, t2, std, ::, time, (, 0, ...",0
4,65908,"[[FuncDef, Decl, FuncDecl, getFrame], [Compoun...",0
...,...,...,...
507314,573218,"[[FuncDef, Decl, FuncDecl, cyber_enable_magic_...",0
507315,573219,[],0
507316,573220,"[[Pointer, FuncDef, Decl, FuncDecl, try_enumer...",0
507317,573221,"[[Pointer, FuncDef, Decl, FuncDecl, find_resto...",0


In [21]:
df_false.code[0]

[['Pointer',
  'FuncDef',
  'Decl',
  'FuncDecl',
  'array_print',
  'array',
  '*',
  'a',
  'int',
  'depth'],
 ['Compound'],
 ['Decl', 'size_t', 'i'],
 ['Decl', 'size_t', 'maxlen'],
 ['Decl', 'int', 'oneline', '1'],
 ['Selection', 'Condition', 'if', 'a', '->', 'used', '>', '5'],
 ['Compound'],
 ['oneline', '=', '0'],
 ['End'],
 ['Iteration',
  'ForExpr',
  'Condition',
  'ForInit',
  'for',
  'i',
  '=',
  '0',
  'i',
  '<',
  'a',
  '->',
  'used',
  '&&',
  'oneline',
  'i',
  '++'],
 ['Compound'],
 ['Decl',
  'Pointer',
  'data_unset',
  '*',
  'du',
  'a',
  '->',
  'data',
  '[',
  'i',
  ']'],
 ['Selection', 'Condition', 'if', '!', 'du', '->', 'is_index_key'],
 ['Compound'],
 ['oneline', '=', '0'],
 ['break'],
 ['End'],
 ['Selection', 'Condition', 'switch', 'du', '->', 'type'],
 ['Compound'],
 ['case', 'TYPE_INTEGER', ':'],
 ['case', 'TYPE_STRING', ':'],
 ['case', 'TYPE_COUNT', ':'],
 ['break'],
 ['default', ':'],
 ['oneline', '=', '0'],
 ['break'],
 ['End'],
 ['End'],
 ['Sele

### Prepare for Word2Vec

In [87]:
## Load train data
del(train)
train = pd.read_pickle('train/train_name_block.pkl')
train = train.reset_index()

In [42]:
def flatten(x):
    return(list(chain.from_iterable(x)))

In [44]:
from itertools import chain
corpus = train.code.apply(flatten)

### Train Word2Vec

In [ ]:
from gensim.models.word2vec import Word2Vec
w2v = Word2Vec(corpus, size=128, workers=16, sg=1, min_count=3)

In [53]:
w2v.save('train/node_w2v_128')

### Load Word2Vec

In [5]:
from gensim.models.word2vec import Word2Vec

w2v = Word2Vec.load('train/node_w2v_128')

In [70]:
import copy
code = copy.deepcopy(train.code[1])

In [6]:
global statement_line
statement_line = -2
def replace(node,i):
    global statement_line
    if type(node) == str:
        code[statement_line][i] = [w2v.wv.vocab.get(node).index] if node in w2v.wv.vocab else [w2v.wv.vectors.shape[0]-1]

    else:
        statement_line+=1

        for ind in range(len(node)):
            replace(node[ind],ind)

In [72]:
w2v.wv.vocab.get('Pointer').index

11

In [73]:
w2v.wv.vectors.shape[0]

867572

In [74]:
w2v.wv.index2word[867571]

'GMT_get_map_interval'

In [ ]:
replace(code,0)
code

### Replace Word with Index from Word2Vec

In [ ]:
train.code[0:5].apply(replace,args=(0,))

In [88]:
for code in train.code:
    statement_line = -2
    replace(code,0)

In [94]:
train.to_pickle('blocks_train.pkl')

In [90]:
del(code)
test = pd.read_pickle('train/test_name_block.pkl')
test = test.reset_index()

for code in test.code:
    statement_line = -2
    replace(code,0)

In [95]:
test.to_pickle('blocks_test.pkl')

In [7]:
val = pd.read_pickle('train/val_name_block.pkl')
val = val.reset_index()

for code in val.code:
    statement_line = -2
    replace(code,0)

In [96]:
val.to_pickle('blocks_val.pkl')

### DO SOME SHIT

In [1]:
import pandas as pd

In [39]:
val = pd.read_pickle('blocks_val.pkl')

In [46]:
val.sample

,index,id,code,label
0,97858,106112,"[[21, 4, 22, 784073, 19, 1486, 19, 13479], [9]...",0
1,4303,4303,"[[11, 21, 4, 22, 867571, 26, 10650, 5, 62], [1...",1
2,101958,110212,"[[11, 21, 4, 22, 867571, 35, 2062, 17, 785], [...",0
3,85037,93291,"[[11, 21, 4, 22, 867571, 26, 56524, 5, 6351, 2...",0
4,30900,39154,"[[11, 21, 4, 22, 867571, 13378, 5, 2179, 39610...",0
...,...,...,...,...
127471,2878,2878,"[[11, 21, 4, 22, 867571, 27, 5, 5, 54, 19, 581...",1
127472,116576,124830,"[[11, 21, 4, 22, 307253, 90684, 5, 25658], [9]...",0
127473,32111,40365,"[[11, 21, 4, 22, 867571, 26, 239236, 5, 51256,...",0
127474,67396,75650,"[[11, 21, 4, 22, 867571, 1640, 5, 295], [9], [...",0


In [30]:
val.code[0]

[[[21], [4], [22], [784073], [19], [1486], [19], [13479]],
 [[9]],
 [[8], [6], [10], [13479]],
 [[9]],
 [[1], [0], [55602], [1486]],
 [[7]],
 [[8], [6], [34]],
 [[9]],
 [[1], [0], [867571], [1486], [78], [867571], [25]],
 [[867571]],
 [[7]],
 [[7]]]

In [28]:
import torch
t = torch.Tensor([[1], [6]])
t.shape

torch.Size([2, 1])

In [29]:
k = torch.Tensor([1,2])
k.shape

torch.Size([2])

In [31]:
k=[1,32]

In [37]:
len(k[0][0])

TypeError: 'int' object is not subscriptable

In [38]:
k[0][0] is -1

TypeError: 'int' object is not subscriptable

In [77]:
import pycparser
import pandas as pd
from prepare_data import get_blocks as func

def tree_to_index(node):
    token = node.token
    result = [token]
    children = node.children
    for child in children:
        result.append(tree_to_index(child))
    return result

def trans2seq(r):
    blocks = []
    func(r, blocks)
    tree = []
    for b in blocks:
        btree = tree_to_index(b)
        tree.append(btree)
    return tree

In [78]:
ast = pycparser.parse_file('myCS/input/test2.c')

In [79]:
trans2seq(ast)

[['FuncDef', ['Decl', ['FuncDecl', ['main', ['int']]]]],
 ['Compound'],
 ['Decl', ['x', ['int']], ['22']],
 ['Decl', ['y', ['int']], ['44']],
 ['If', ['==', ['x'], ['23']]],
 ['Compound'],
 ['=', ['x'], ['2']],
 ['End'],
 ['Compound'],
 ['=', ['y'], ['314']],
 ['End'],
 ['While', ['<', ['x'], ['21']]],
 ['Compound'],
 ['FuncCall', ['printf'], ['ExprList', ['"value of x: %d\\n"'], ['x']]],
 ['++', ['x']],
 ['End'],
 ['=', ['x'], ['+', ['x'], ['x']]],
 ['FuncCall', ['printf'], ['ExprList', ['"%d"'], ['x']]],
 ['Return', ['0']],
 ['End']]

In [3]:
stmt = ['=', 'x', '2', 'z', 'y', 'p','m','n','k', 'q','out', 'a','c','d','f']
lvl  = [10, 11, 11, 11, 12, 12, 12, 12,12,12,12, 13, 13, 13, 14]

### In manual 

In [4]:
out = []

# level 0
out.append(stmt[0])

# level 1
out.append([stmt[1]])
out.append([stmt[2]])
out.append([stmt[3]])

# level 2
out[1].append([stmt[4]])
out[1].append([stmt[5]])
out[2].append([stmt[6]])

# level 3
out[1][1].append([stmt[7]])

out

['=', ['x', ['y', ['n']], ['p']], ['2', ['m']], ['z']]

### In auto

In [5]:
out = [[]]
count_level = -1
curr = []
full = True

for i in range(0,len(lvl)):
    
    if i == 0:
        # level 0
        out[0].append(stmt[0])
    else:
        if lvl[i]!=lvl[i-1]:
            count_level +=1
            curr = get_shit(count_level)
            
            if count_level ==0:
                curr.append([stmt[i]])
            else:
                curr[0].append([stmt[i]])
                
        else:
            if count_level ==0:
                curr.append([stmt[i]])
            else:
                full = True
                for j in range(0,len(curr)):
                    if len(curr[j])<3:
                        curr[j].append([stmt[i]])
                        full = False
                        break
                        
                if full == True:
                    for j in range(0,len(curr)):
                        if len(curr[j])<4:
                            curr[j].append([stmt[i]])
                            break
out[0]

['=',
 ['x', ['y', ['a', ['f']], ['c']], ['p', ['d']], ['out']],
 ['2', ['m'], ['n']],
 ['z', ['k'], ['q']]]

In [6]:
### return list of that 

def get_shit(count_level):
    
    if count_level == 0:
        return(out[0])
    
    if count_level == 1:
        return(out[0][1:])
    
    if count_level == 2:
        temp = []
        for ls1 in out[0][1:]:
            for ls2 in ls1[1:]:
                temp.append(ls2)
        return(temp)
        
    
    if count_level == 3:
        temp = []
        for ls1 in out[0][1:]:
            for ls2 in ls1[1:]:
                for ls3 in ls2[1:]:
                    temp.append(ls3)
        return(temp)
    
    if count_level == 4:
        temp = []
        for ls1 in out[0][1:]:
            for ls2 in ls1[1:]:
                for ls3 in ls2[1:]:
                    for ls4 in ls3[1:]:
                        temp.append(ls4)
        return(temp)
    
    if count_level == 5:
        temp = []
        for ls1 in out[0][1:]:
            for ls2 in ls1[1:]:
                for ls3 in ls2[1:]:
                    for ls4 in ls3[1:]:
                        for ls5 in ls4[1:]:
                            temp.append(ls5)
        return(temp)
    
    if count_level == 6:
        temp = []
        for ls1 in out[0][1:]:
            for ls2 in ls1[1:]:
                for ls3 in ls2[1:]:
                    for ls4 in ls3[1:]:
                        for ls5 in ls4[1:]:
                            for ls6 in ls5[1:]:
                                temp.append(ls6)
        return(temp)
    
    if count_level == 7:
        temp = []
        for ls1 in out[0][1:]:
            for ls2 in ls1[1:]:
                for ls3 in ls2[1:]:
                    for ls4 in ls3[1:]:
                        for ls5 in ls4[1:]:
                            for ls6 in ls5[1:]:
                                for ls7 in ls6[1:]:
                                    temp.append(ls7)
        return(temp)
    
    if count_level == 8:
        temp = []
        for ls1 in out[0][1:]:
            for ls2 in ls1[1:]:
                for ls3 in ls2[1:]:
                    for ls4 in ls3[1:]:
                        for ls5 in ls4[1:]:
                            for ls6 in ls5[1:]:
                                for ls7 in ls6[1:]:
                                    for ls8 in ls7[1:]:
                                        temp.append(ls8)
        return(temp)
    
    if count_level == 9:
        temp = []
        for ls1 in out[0][1:]:
            for ls2 in ls1[1:]:
                for ls3 in ls2[1:]:
                    for ls4 in ls3[1:]:
                        for ls5 in ls4[1:]:
                            for ls6 in ls5[1:]:
                                for ls7 in ls6[1:]:
                                    for ls8 in ls7[1:]:
                                        for ls9 in ls8[1:]:
                                            temp.append(ls9)
        return(temp)
    
    if count_level == 10:
        temp = []
        for ls1 in out[0][1:]:
            for ls2 in ls1[1:]:
                for ls3 in ls2[1:]:
                    for ls4 in ls3[1:]:
                        for ls5 in ls4[1:]:
                            for ls6 in ls5[1:]:
                                for ls7 in ls6[1:]:
                                    for ls8 in ls7[1:]:
                                        for ls9 in ls8[1:]:
                                            for ls10 in ls9[1:]:
                                                temp.append(ls10)
        return(temp)
    

In [25]:
tree_lvl = pd.Series(lvl).unique()
tree_lvl 

array([10, 11, 12, 13])

In [95]:
k = '[\'x\', \'int\', \'=\', \'x\', \'22\']'


In [96]:
import ast
ast.literal_eval(k)

['x', 'int', '=', 'x', '22']

In [122]:
np.where(lvl)

(array([0, 1, 2, 3, 4, 5, 6, 7]),)

In [ ]:
ast